# Deploying Machine Learning Models with FastAPI

This notebook will introduce you to the key aspects of deploying machine learning models using FastAPI. We'll explore setting up a FastAPI server, handling file uploads, and integrating a pre-trained machine learning model for image classification. You'll learn about the essential components such as route creation, asynchronous request handling and forming JSON responses

### Installation Guide

```bash
pip install fastapi uvicorn


### Introduction to FastAPI

What is FastAPI?
FastAPI is a modern Python web framework designed explicitly for building high-performance RESTful APIs.
    

```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def hello():
    return {"message": "Hello, this is your first API with FastAPI"}


Run the above example from the command line using:

```bash
uvicorn fastapi_hello_world:app --host 0.0.0.0 --port 80


### Building Your First Machine Learning API

#### FastAPI App Structure
The basic structure of a FastAPI application is straightforward:
- **App creation**: An instance of `FastAPI()`.
- **Endpoint definition**: Using decorators to define functions for particular HTTP methods at specific API paths.

```python
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
import torch
from torchvision import models, transforms
from PIL import Image
import io

app = FastAPI()

# Load the pre-trained model
model = models.resnet18(pretrained=True)
model.eval()  # Set model to evaluation mode

# Define image transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    image_data = await file.read()
    image = Image.open(io.BytesIO(image_data))
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        output = model(image)
        predicted_index = output.argmax(1).item()
    
    return JSONResponse(content={"predicted_class": predicted_index})

This code snippet illustrates how to define a more complex POST endpoint in FastAPI that handles image data for ML predictions.

- `@app.post("/predict/")`: Defines a route that listens for POST requests, where users can send images for classification.
- `async def predict(file: UploadFile = File(...))`: The endpoint function is asynchronous, which improves performance by handling requests concurrently. It accepts an uploaded file through the request.
- `await file.read()`: Asynchronously reads the uploaded file's data, efficient for handling I/O operations.
- `JSONResponse`: Returns the predicted class in a JSON format, making the API suitable for integration with other services or applications that consume JSON.


### Test the API Using Python Requests
With the app running, let's use the requests library to send an image to the FastAPI application and receive a prediction:

In [2]:
import requests

# URL of the FastAPI endpoint
url = 'http://localhost:8000/predict/'

# Path to the image file
file_path = 'imgs/cat.jpeg' 

# Open the image file in binary mode
with open(file_path, 'rb') as f:
    # Prepare the request payload as a dictionary
    files = {'file': (file_path, f, 'image/jpeg')}
    
    # Send the POST request
    response = requests.post(url, files=files)

# Print the response
print(response.json())

{'predicted_class': 281}


- `files = {'file': (file_path, f, 'image/jpeg')}`: Constructs a dictionary for the file payload. This dictionary is formatted to be compatible with HTTP multipart/form-data encoding, which is typically used for file uploads
- `response = requests.post(url, files=files)`: Sends a POST request to the specified url with the file data. The files parameter is used to indicate that a file is included in the request.
- `response.json()`: Parses the JSON response from the FastAPI server. This method converts the JSON returned by the server into a Python dictionary, allowing easy access to the prediction results or any other data returned by the server.

### Dockerizing Your API

#### Containerization and Docker
Containerization involves encapsulating an application and its environment into a container that can run on any Docker-enabled system. This ensures consistency across different development and production environments.

<img src="./imgs/docker_deployment_1.png" alt="drawing" width="800"/>

<img src="./imgs/docker_deployment.png" alt="drawing" width="800"/>

#### Dockerfile for FastAPI
To dockerize your FastAPI application, you need a Dockerfile that specifies the environment, dependencies, and how to run your app.

```Docker
# Use an official PyTorch runtime as a parent image
FROM pytorch/pytorch:1.9.0-cuda11.1-cudnn8-runtime

# Set the working directory in the container
WORKDIR /app

# Copy the current directory contents into the container at /app
COPY . /app

# Install any needed packages specified in requirements.txt
RUN pip install fastapi uvicorn Pillow torchvision python-multipart

# Run app.py when the container launches
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "80"]

- The Dockerfile begins with `FROM pytorch/pytorch:1.9.0-cuda11.1-cudnn8-runtime`, specifying a base image with Pytorch 
- The `WORKDIR` command sets the working directory inside the container.
- The `COPY . /app` command copies the application files from the current directory on the host into the `/app` directory in the container. 
- Following this, `RUN pip install` installs the Python dependencies
- `EXPOSE 80` makes port 80 available outside the container, and the `CMD` command specifies the command to run the FastAPI app using Uvicorn, a lightning-fast ASGI server, on port 80. This setup ensures that the FastAPI application can be easily deployed and run in any environment that supports Docker.

#### Building and Running a Docker Image
To build and run your FastAPI application inside a Docker container, use the following commands:

```bash
docker build -t myfastapiapp .
docker run -p 8000:80 myfastapiapp

This segment outlines the commands needed to build and run a Docker image containing a FastAPI application. 
- `docker build -t myfastapiapp .` command builds a Docker image from the Dockerfile in the current directory and tags it as `myfastapiapp`.
- `docker run -p 8000:80 myfastapiapp` command runs the built Docker image as a container. The `-p 8000:80` option maps port 80 inside the container (where the FastAPI app is running) to port 8000 on the host machine. This mapping allows the FastAPI application to be accessed from the host machine's port 8000, facilitating easy interaction with the API during development or production.

### Test the API again using Python Requests
With the app running, let's use the requests library to send an image to the FastAPI application and receive a prediction:

In [3]:
import requests

# URL of the FastAPI endpoint
url = 'http://localhost:8000/predict/'

# Path to the image file
file_path = 'imgs/cat.jpeg' 

# Open the image file in binary mode
with open(file_path, 'rb') as f:
    # Prepare the request payload as a dictionary
    files = {'file': (file_path, f, 'image/jpeg')}
    
    # Send the POST request
    response = requests.post(url, files=files)

# Print the response
print(response.json())

{'predicted_class': 281}


### Dockerizing Your API

#### Containerization and Docker
Containerization involves encapsulating an application and its environment into a container that can run on any Docker-enabled system. This ensures consistency across different development and production environments.

## Conclusion

In this this workshop, we have explored the process of deploying machine learning models using FastAPI, from setting up a basic API to integrating and containerizing the application using Docker for consistent, scalable deployments.

We covered a wide range of topics essential for anyone looking to deploy their machine learning models into production environments:

- **FastAPI's Essentials**: We began with the basics of FastAPI, appreciating its speed, ease of use, and features like automatic data validation and documentation which make it an excellent choice for ML deployments.
- **Machine Learning Integration**: We demonstrated how to integrate a pre-trained machine learning model into a FastAPI application, handling both GET and POST requests.
- **Dockerization**: We containerized our FastAPI application using Docker, which not only simplifies deployment across different environments but also aids in achieving consistency and scalability.
- **Production Deployment**: Finally, we discussed best practices for deploying our application to production, including considerations for scaling, security, and monitoring.
